# Creating an Agent via boto3 SDK
Auther: xiaoqunn@amazom.com (GCR ALML GenAI SSA)

Creating an Agent via boto3 SDK
* Step 0. Pre-requests
  + Prepare python environments.
  + Open AWS SES service: https://us-east-1.console.aws.amazon.com/ses/home?region=us-east-1#/get-set-up
* Step 1. Create Lambda function & upload relevant files to s3
  + Step 1.1 Create lambda roles
  + Step 1.2 Create lambda function
  + Step 1.3 Upload relavent files
  + Step 1.4 Create lambda layer & add layer
  + Step 1.5 Create bedrock agent role
* Step 2. Create an Agent
* Step 3. Create Action group
* Step 4. Associate knowledge base(Optional)
* Step 5. Prepare agent and create agent alias
* Step 6. Invoking Agent
* Step 8. Test Agent
* Step 9. Delete resourse

### Step 0. Pre-requests.

#### Prepare python environments.

In [ ]:
!pip install boto3

#### Open AWS SES service

url: https://us-east-1.console.aws.amazon.com/ses/home?region=us-east-1#/get-set-up

In [ ]:
%%HTML
<img src="../imgs/Picture1.png", width=600>

This email will automaticlly add to lambda Environment Variables, Use as sender and recipient address(if you want to use another email as recipient, you should verify it too).

### Step 1. Create Lambda function & upload relevant files

In [ ]:
import boto3
import uuid
import logging
import json

from uuid import uuid4
from botocore.exceptions import ClientError
from utils import progress_bar


logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
credentials = boto3.Session().get_credentials()
account_id = "" # Your Account ID
aws_access_key_id = credentials.access_key # you can modify to your access_key 
aws_secret_access_key = credentials.secret_key  # you can modify to your secret_key 
region = "us-east-1"
iam = boto3.resource("iam", region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
lambda_client= boto3.client("lambda", region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)



#### Step 1.1 Create lambda roles

In [ ]:
# create lambda role for bedrock agent
from utils import create_role, create_policy, attach_policy, get_role
lammbda_role_name = f"bedrock-agent-lambda-demo-{uuid4()}"
lambda_agent_demo_role = create_role(
    iam,
    lammbda_role_name,
    ["lambda.amazonaws.com", "bedrock.amazonaws.com"]
    )
print(f"{lambda_agent_demo_role.name}, {lambda_agent_demo_role.arn}")

In [ ]:
# attach full access policy
full_access_lambda_policy = create_policy(
    iam,
    f"full-demo-policy-{uuid4()}",
    "Full access for agent lambda demo.",
    "*",
    "*"
)
attach_policy(
    iam,
    lammbda_role_name,
    full_access_lambda_policy.arn
)
progress_bar(5)

#### Step 1.2 Create lambda function

In [ ]:
from utils import create_function, create_deployment_package

# Create lambda function
bedrock_agent_lambda_demo = get_role(iam, lammbda_role_name)
print("Zipping the Python script into a deployment package...")

lambda_name = "invoice_test2"
deployment_package = create_deployment_package(
    "../invoice_lambda.py",
    f"{lambda_name}.py"
)
print(f"...and creating the {lambda_name} Lambda function.")

lambda_function_arn = create_function(
    lambda_client,
    lambda_name,
    f"{lambda_name}.lambda_handler",
    lambda_agent_demo_role,
    deployment_package
)
print(f"Successfully create the {lambda_name} Lambda function.")

#### Step 1.3 Upload relavent files

In [ ]:
# Zip lambda layer
!zip -r ../invoice_lambda_layer.zip ../python

In [ ]:
import os
import random
bucket_name = f"invoice-agent-demo-{uuid4()}"
s3_client = boto3.client("s3", region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Upload demo relavant files
file_path = "../conf"
file_list = os.listdir(file_path)
print(file_list)
response = s3_client.create_bucket(Bucket=bucket_name)
for file_name in file_list:
    upload_schema = s3_client.upload_file(os.path.join(file_path, file_name), bucket_name, file_name)


# Upload lambda layer
lambda_layer = "invoice_lambda_layer.zip"
respose = s3_client.upload_file("../"+lambda_layer, bucket_name, lambda_layer)
print(response)

# Upload API schema file
invoice_schema = "invoic_schema.json"
respose = s3_client.upload_file("../invoice_service_schema.json", bucket_name, invoice_schema)
print(response)
s3_schema_path = f"arn:aws:s3:::{bucket_name}/{invoice_schema}"

#### Step 1.4 Create lambda layer & add layer

How to create a Python layer for your dependencies, reference link: https://docs.aws.amazon.com/lambda/latest/dg/python-package.html

In [ ]:
LayerName = "invoice-demo-layer2"
response = lambda_client.publish_layer_version(
    LayerName=LayerName,
    Description='an layer for agent lambda function',
    Content={
        'S3Bucket': bucket_name,
        'S3Key': lambda_layer,
    },
    CompatibleRuntimes=[
        'python3.10',
    ],
    CompatibleArchitectures=[
        'x86_64',
    ]
)

layer_arn = response["LayerArn"]
layer_version_arn = response['LayerVersionArn']
layer_version = int(response['Version'])
print(f"layer_arn: {layer_arn}\nlayer_version: {layer_version}\nlayer_version_arn: {layer_version_arn}")
progress_bar(5)

In [ ]:
# Add BUCKET_NAME parameter and lambda layer
sender_email = "xiaoqunn@amazon.com" # Replace by your verified email
lambda_client.update_function_configuration(
    FunctionName=lambda_name, Environment={"Variables": {"BUCKET_NAME": bucket_name, "SENDER": sender_email}}, Layers=[layer_version_arn]
)

In [ ]:
#Add permission

# account_id = "" # Your Account ID
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allowinvoke',
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=f"arn:aws:bedrock:us-east-1:{account_id}:agent/*", 
    SourceAccount=account_id
)
print(response)

#### Step 1.5 Create bedrock agent role

In [ ]:
# Create Bedrock Agent role
# Role name must startwith "AmazonBedrockExecutionRoleForAgents_" 
bedrock_agent_role_name = "AmazonBedrockExecutionRoleForAgents_demo2"
bedrock_agent_role = create_role(
        iam,
        bedrock_agent_role_name,
        ["bedrock.amazonaws.com"]
    )
print(bedrock_agent_role.arn)

In [ ]:
# Create s3 policy and bedrock invoke policy
s3_schema_policy = create_policy(
    iam,
    f"invoice-schema-policy-{uuid4()}",
    "Policy for IAM demonstration.",
    ["s3:GetObject"], 
    s3_schema_path
)
bedrock_agent_invoke_demo_policy = create_policy(
    iam,
    f"bedrock_agent_invoke_demo_policy-{uuid4()}",
    "Policy for IAM demonstration.",
    "bedrock:InvokeModel",
    [f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-v2",
     f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-v2:1",
     f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-instant-v1",
     f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0",
     f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-3-haiku-20240307-v1:0"]
)



In [ ]:
# attach policy to Bedrock Agent role
attach_policy(
    iam,
    bedrock_agent_role.name,
    s3_schema_policy.arn
)
attach_policy(
    iam,
    bedrock_agent_role.name,
    bedrock_agent_invoke_demo_policy.arn
)



### Step 2. Create an Agent

In [ ]:
client = boto3.client("bedrock-agent", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

agent_name = "invoice-test-demo2"

# Claude 3 Sonnet instruction
instructions = """You are an AI assistant with the capability help user with using given tools. If the user asks to issue an invoice, the first thing you need to do is collect all the required information, then call the functions."""

# Claude V2 instruction
# instructions = """You are a friendly invoice assistant. When greeted, answer user with "I'm an invoice assistant". Through the "InvoiceService" action group, you can offer invoice services. When generating an invoice, first collect all required invoice information from user. Then generate invoice preview information for the user's reference and return text_info from the function result to user. Confirm with user if they want to proceed with generating the actual invoice. If user confirms, use function to generate an invoice formally and return the downloadUrl from the function result to user. This allows user to download the invoice. If user indicates the information is incorrect, ask them to provide corrected information and generate preview infomation again. Finally confirm if the user needs the invoice sent to a designated email address, if so, email the invoice file to the address provided."""

response = client.create_agent(
    agentName = agent_name,
    agentResourceRoleArn = bedrock_agent_role.arn,
    description = "invocie test",
    idleSessionTTLInSeconds = 1800,
    foundationModel = "anthropic.claude-3-sonnet-20240229-v1:0",
    instruction = instructions,
)
agent_id = response['agent']['agentId']
progress_bar(5)

### Step 3. Create Action group

In [ ]:
action_group_name = "InvoiceService"
s3_bucket = bucket_name
s3_object_key = invoice_schema
description = "An invocie service"

response = client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT', 
    actionGroupExecutor={
        'lambda': lambda_function_arn
        },
    actionGroupName = action_group_name,
    apiSchema={
        's3': {
            's3BucketName': s3_bucket,
            's3ObjectKey': s3_object_key
            }
    },
    description = description
)

### Step 4. Associate knowledge base(Optional)
<strong> You can create a knowledge base use "knowledge_base.ipynb" </strong>

In [ ]:
# You can create a knowledge base use "knowledge_base.ipynb"
# If you already have a knoweledge base, and you want to integrate it in agent, uncomment it 

# knowledge_base_arn = f"arn:aws:bedrock:us-east-1:{account_id}:knowledge-base/*"

# kb_retrive_policy = create_policy(
#     iam,
#     "invoice-agent-kb-demo-policy",
#     "Policy for agent kb retreive.",
#     ["bedrock:Retrieve"],
#     [knowledge_base_arn] 
# )

# attach_policy(
#     iam,
#     bedrock_agent_role.name,
#     kb_retrive_policy.arn 
# )

# response = client.associate_agent_knowledge_base(
#     agentId=agent_id,
#     agentVersion='DRAFT',
#     description='Use this knowledge base whenever question relate to invoice or issurance.',
#     knowledgeBaseId='<KNOWLEDGEBASE_ID>'
# )

### Step 5. Prepare agent and create agent alias

In [ ]:
import time
agent_alias_name = "demo_test"

client = boto3.client("bedrock-agent", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
response = client.prepare_agent(agentId=agent_id)
progress_bar(10)

Notice: Need to wait a minute for prepare agent

In [ ]:
agent_alias_description = "init version"
# agent_alias_description = "version2"
agent_alias = client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name,
    description=agent_alias_description
)
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

### Step 6. Invoking Agent

In [ ]:
# Need change to "bedrock-agent-runtime" 
client = boto3.client("bedrock-agent-runtime", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

sessionid = str(uuid.uuid1())
enable_trace:bool = True 

def invoke(question: str, sessionid: str, agent_id: str, agent_alias_id: str, enable_trace=False):
    final_answer = ""
    response = client.invoke_agent(
        inputText=question, # 输入文本
        agentId=agent_id,  # 创建的 Agent 的 ID
        agentAliasId=agent_alias_id, # 创建的 Agent alias id
        sessionId=sessionid, # 当前会话的session id
        enableTrace=enable_trace # 是否打开 trace
    )
    event_stream = response['completion']
    try:
        for event in event_stream:        
            # print(event)
            if 'chunk' in event:
                data = event['chunk']['bytes']
                final_answer = data.decode('utf8')
                print(f"Final answer ->\n{final_answer}") 
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                trace = event['trace']
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    
    return final_answer

### Step 9. Test
You can also test in console

In [ ]:
sessionid = str(uuid.uuid1())
enable_trace:bool = True 

In [ ]:
question = "Hello"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

In [ ]:
question = "我要开发票"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

In [ ]:
question = "91440300MA5FAE9E4P, 华韵公司，小麦，1010101020000000000，9000"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

In [ ]:
question = "确认"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

In [ ]:
question = "发送发票到邮箱，xiaoqunn@amazon.com"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

## Step 9. Delete resource

In [ ]:
from utils import teardown

# delete Agent
client = boto3.client("bedrock-agent", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
response = client.delete_agent(
    agentId=agent_id,
    skipResourceInUseCheck=True
)

# delete lambda function
response = lambda_client.delete_function(
    FunctionName=lambda_name,
)
print(response)

# delete lambda layer
response = lambda_client.delete_layer_version(
    LayerName=LayerName,
    VersionNumber=layer_version
)
print(response)


# delete S3 buket
s3_resource = boto3.resource("s3")
bucket = s3_resource.Bucket(bucket_name)
bucket.objects.delete()
bucket.delete()
print(f"Emptied and deleted bucket {bucket.name}.\n")

# delete role and policy
teardown(iam, [lambda_agent_demo_role, bedrock_agent_role])
